In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
datetime.now()

datetime.datetime(2020, 3, 3, 6, 57, 6, 171099)

In [3]:
url = 'https://www.health.gov.au/news/coronavirus-update-at-a-glance'
content = requests.get(url).content

In [4]:
soup = BeautifulSoup(content)
data = soup.findAll(text=re.compile('As .+')) + \
        soup.findAll(text=re.compile('.+ in (?:Queensland|Victoria|(New South Wales)|(South Australia)|(Western Australia))')) 
data = ' '.join(data)
data

'As at 06:30\xa0hrs on 3\xa0March 2020, we have 33\xa0confirmed cases of coronavirus (COVID-19) in Australia.'

In [5]:
cases_str = unicodedata.normalize("NFKD", data)

In [6]:
cases_str

'As at 06:30 hrs on 3 March 2020, we have 33 confirmed cases of coronavirus (COVID-19) in Australia.'

In [7]:
date_time_str = re.search('As .+ (\d+ \w+ 202\d)' ,cases_str).group(1)
date_time_str

'3 March 2020'

In [8]:
data_date_time = datetime.strptime(date_time_str, '%d %B %Y').strftime('%Y-%m-%d')
data_date_time

'2020-03-03'

In [9]:
m = re.search('have (\d+) .* cases .+ in Australia', cases_str)
australia = int(m.group(1))


In [10]:
# m = re.search('(\d+) in New South Wales', cases_str)
# nsw = int(m.group(1))


In [11]:
# m = re.search('(\d+) in Victoria', cases_str)
# victoria = int(m.group(1))

In [12]:
# m = re.search('(\d+) in South Australia', cases_str)
# sa = int(m.group(1))

In [13]:
# m = re.search('(\d+) in Queensland', cases_str)
# qld = int(m.group(1))

In [14]:
# m = re.search('(\d+) in Western Australia', cases_str)
# wa = int(m.group(1))


### Dataframe

In [15]:
# df = pd.DataFrame({'date': data_date_time, 'australia': australia, 'nsw': nsw, 'victoria': victoria, 'sa': sa, 'qld': qld, 'wa': wa}, index=[0])
df = pd.DataFrame({'date': data_date_time, 'australia': australia}, index=[0])

In [16]:
df

,date,australia
0,2020-03-03,33


### Save to file

In [17]:
from pathlib import Path
csv_file = 'data/australia-cases.csv'

if Path(csv_file).exists():
    # read out the old data
    old_df = pd.read_csv(csv_file)
    df = df.append(old_df)
df.drop_duplicates(subset=['date'], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [18]:
df.sort_values(by=['date'], inplace=True)
df.to_csv(csv_file, index=False)
df

,australia,date,nsw,qld,sa,victoria,wa
0,12,2020-02-04,4.0,2.0,2.0,4.0,NaN
1,13,2020-02-05,4.0,3.0,2.0,4.0,NaN
2,14,2020-02-06,4.0,4.0,2.0,4.0,NaN
3,15,2020-02-07,4.0,5.0,2.0,4.0,NaN
4,15,2020-02-08,4.0,5.0,2.0,4.0,NaN
5,15,2020-02-09,4.0,5.0,2.0,4.0,NaN
6,15,2020-02-10,4.0,5.0,2.0,4.0,NaN
7,15,2020-02-11,4.0,5.0,2.0,4.0,NaN
8,15,2020-02-12,4.0,5.0,2.0,4.0,NaN
9,15,2020-02-13,4.0,5.0,2.0,4.0,NaN
